In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = tf.keras.layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=(bias is not None),
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    new_conv_layer.build(conv_layer.input.shape)
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, new_filters = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer and Next Convolutional Layer
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# **FIX: Explicitly define input shape for conv2_pruned**
pruned_conv2 = layers.Conv2D(
    filters=64, kernel_size=(3, 3), activation='relu',
    input_shape=(13, 13, new_filters),  # **Ensure correct input shape**
    name="conv2_pruned"
)

pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# **FIX: Ensure Flatten layer adapts to new shape**
pruned_flatten = layers.Flatten(name="flatten_pruned")

# 8️⃣ Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),  # **Ensure input shape is set**
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    pruned_flatten,
    model.layers[5],  # Dense
    model.layers[6],  # Output Layer
])

# 9️⃣ Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))  # **Ensure layers are built**
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



🔹 Baseline Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)                │ (None, 26, 26, 7)           │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1_pruned (MaxPooling2D)          │ (None, 13, 13, 7)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_pruned (Conv2D)                │ (None, 11, 11, 64)          │           4,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2_pruned (MaxPooling2D)          │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_pruned (Flatten)             │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 210,384 (821.81 KB)

 Trainable params: 210,384 (821.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.9540 - loss: 0.1673 - val_accuracy: 0.9880 - val_loss: 0.0381
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9902 - loss: 0.0310 - val_accuracy: 0.9869 - val_loss: 0.0425
✅ Pruned Model Accuracy: 0.9869

📊 Accuracy Before Pruning: 0.9883
📊 Accuracy After Pruning: 0.9869


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=True,
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    input_depth = kernel.shape[2]  # Preserve input depth
    new_conv_layer.build((None, None, None, input_depth))
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, filters_conv1 = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer to Match the New Output
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# 8️⃣ Prune the Second Convolutional Layer to Match `conv1_pruned` Output Depth
original_conv2 = model.layers[2]  # Get the second Conv2D layer
pruned_conv2, filters_conv2 = prune_conv_layer(original_conv2, prune_ratio=0.5)

# 🔍 Fix Input Depth Mismatch
pruned_conv2 = layers.Conv2D(
    filters=filters_conv2,
    kernel_size=original_conv2.kernel_size,
    strides=original_conv2.strides,
    padding=original_conv2.padding,
    activation=original_conv2.activation,
    use_bias=True,
    name=original_conv2.name + "_pruned"
)

# Ensure the input shape matches the output of `pruned_conv1`
pruned_conv2.build((None, None, None, pruned_conv1.filters))

# 9️⃣ Modify Pooling Layer to Match the New Output
pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# 🔟 Compute the Flattened Output Shape Dynamically
dummy_input = np.random.randn(1, 28, 28, 1).astype(np.float32)  # Sample input batch
intermediate_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten()
])

# Get the output shape of the Flatten layer dynamically
dummy_output = intermediate_model.predict(dummy_input)
flattened_dim = dummy_output.shape[1]  # Correct dimension for Dense layer

# 🔟 Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten(),
    layers.Dense(128, activation='relu', name="dense1_pruned"),  # Fixing input shape
    layers.Dense(10, activation='softmax', name="output")  # Output Layer
])

# 🔟 Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")



🔹 Baseline Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)                │ (None, 26, 26, 7)           │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1_pruned (MaxPooling2D)          │ (None, 13, 13, 7)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_pruned (Conv2D)                │ (None, 11, 11, 32)          │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2_pruned (MaxPooling2D)          │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1_pruned (Dense)                │ (None, 128)                 │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,936 (413.81 KB)

 Trainable params: 105,936 (413.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.9104 - loss: 0.3113 - val_accuracy: 0.9824 - val_loss: 0.0527
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9835 - loss: 0.0534 - val_accuracy: 0.9842 - val_loss: 0.0464
✅ Pruned Model Accuracy: 0.9842

📊 Accuracy Before Pruning: 0.9884
📊 Accuracy After Pruning: 0.9842


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Create the Original Model
def create_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, (3, 3), activation='relu', name="conv1")(inputs)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', name="conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    x = layers.Flatten(name="flatten")(x)
    x = layers.Dense(128, activation='relu', name="dense1")(x)
    outputs = layers.Dense(10, activation='softmax', name="output")(x)

    return keras.Model(inputs=inputs, outputs=outputs, name="original_model")

# 2️⃣ Load Dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔹 Check layer structure
print(model.summary())

# 4️⃣ Variance-Based Pruning Functions
def compute_filter_variance(conv_layer_name, sample_batch):
    """Computes the variance of activations for each filter given a sample batch."""
    layer_output = model.get_layer(conv_layer_name).output
    intermediate_model = keras.Model(inputs=model.input, outputs=layer_output)

    activations = intermediate_model.predict(sample_batch)
    filter_variance = np.var(activations, axis=(0, 1, 2))  # Variance over spatial dimensions
    return filter_variance

def prune_by_variance(conv_layer_name, prune_ratio=0.5, sample_batch=None):
    """Prunes filters with the lowest activation variance from a Conv2D layer."""
    conv_layer = model.get_layer(conv_layer_name)
    kernel, bias = conv_layer.get_weights()

    filter_variance = compute_filter_variance(conv_layer_name, sample_batch)
    num_filters = kernel.shape[-1]

    # Determine filters to KEEP based on highest variance
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(filter_variance)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=(bias is not None),
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights
    new_conv_layer.build(conv_layer.input.shape)
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 5️⃣ Apply Variance-Based Pruning
sample_batch = x_train[:100]  # Select a small subset for variance computation
pruned_conv1, new_filters = prune_by_variance("conv1", prune_ratio=0.5, sample_batch=sample_batch)

# 6️⃣ Modify the next Conv2D layer (`conv2`) to accept the new number of filters
original_conv2 = model.get_layer("conv2")
modified_conv2 = layers.Conv2D(
    filters=original_conv2.filters,
    kernel_size=original_conv2.kernel_size,
    strides=original_conv2.strides,
    padding=original_conv2.padding,
    activation=original_conv2.activation,
    name="conv2_modified"
)

# 7️⃣ Rebuild Model with Pruned Layer
inputs = keras.Input(shape=(28, 28, 1))
x = pruned_conv1(inputs)
x = model.get_layer("pool1")(x)
x = modified_conv2(x)  # Use modified conv2 to match pruned output
x = model.get_layer("pool2")(x)
x = model.get_layer("flatten")(x)
x = model.get_layer("dense1")(x)
outputs = model.get_layer("output")(x)

new_model = keras.Model(inputs=inputs, outputs=outputs, name="pruned_model")

# 8️⃣ Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 9️⃣ Evaluate Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"\n📊 Accuracy After Variance-Based Pruning: {pruned_accuracy:.4f}")


Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9022 - loss: 0.3129 - val_accuracy: 0.9811 - val_loss: 0.0601
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9862 - loss: 0.0458 - val_accuracy: 0.9863 - val_loss: 0.0404


Model: "original_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 675,104 (2.58 MB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 450,070 (1.72 MB)

None
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9608 - loss: 0.1393 - val_accuracy: 0.9878 - val_loss: 0.0375
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9901 - loss: 0.0313 - val_accuracy: 0.9890 - val_loss: 0.0313

📊 Accuracy After Variance-Based Pruning: 0.9890
